# SECTION 1 : CONFIGURATION ET IMPORTS




### 1. Imports 

In [1]:
# standars 
import os
import sys
import yaml
import random
import shutil
from pathlib import Path
from collections import Counter
import pickle
import json
import time
from datetime import datetime


#scientifique 
import numpy as np
import pandas as pd

print(f" NumPy version: {np.__version__}")
print(f" Pandas version: {pd.__version__}")


# traitement d'images
from PIL import Image

try:
    import cv2
    print(f" PIL  importé")
    print(f" OpenCV version: {cv2.__version__}")
except ImportError:
   
    cv2 = None


#visualisation 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f" Matplotlib version: {matplotlib.__version__}")
print(f" Seaborn version: {sns.__version__}")


#Machine learning 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


#utilitaires 
from tqdm import tqdm

from pathlib import Path
import numpy as np
import random



c:\Users\HP\anaconda3\Lib\site-packages\pandas\core\computation\expressions.py:22: UserWarning: Pandas requires version '2.10.2' or newer of 'numexpr' (version '2.10.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


 NumPy version: 2.4.1
 Pandas version: 3.0.0
 Matplotlib version: 3.10.8
 Seaborn version: 0.13.2


### 2. Chargement 

In [2]:

CONFIG_PATH = Path(r"A:\Mes documents\CNN\config.yaml")

if not CONFIG_PATH.exists():
    raise FileNotFoundError(f" Fichier config.yaml introuvable: {CONFIG_PATH}")

with open(CONFIG_PATH, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print(" Configuration chargée a ")
print()

 Configuration chargée a 



### 3. Définition des constantes globales

In [3]:


# Chemins principaux
BASE_DIR = Path(config['paths']['base_dir'])
RAW_DATA_PATH = BASE_DIR / config['paths']['data']['raw']
TEST_DATA_PATH = BASE_DIR / config['paths']['data']['test_raw']
BALANCED_DATA_PATH = BASE_DIR / config['paths']['data']['balanced']
PROCESSED_TRAIN_PATH = BASE_DIR / config['paths']['data']['processed']['train']
PROCESSED_VAL_PATH = BASE_DIR / config['paths']['data']['processed']['validation']
PROCESSED_TEST_PATH = BASE_DIR / config['paths']['data']['processed']['test']

# Chemins pour les modèles
MODELS_DIR = BASE_DIR / config['paths']['models']

# Chemins pour outputs 
if isinstance(config['paths']['outputs'], dict):
    # Ancien format (dict)
    OUTPUTS_DIR = BASE_DIR / 'outputs'
    FIGURES_DIR = BASE_DIR / config['paths']['outputs']['figures']
    REPORTS_DIR = BASE_DIR / config['paths']['outputs']['reports']
    PREDICTIONS_DIR = BASE_DIR / config['paths']['outputs']['predictions']
else:
    # Nouveau format (string)
    OUTPUTS_DIR = BASE_DIR / config['paths']['outputs']
    FIGURES_DIR = OUTPUTS_DIR / 'figures'
    REPORTS_DIR = OUTPUTS_DIR / 'reports'
    PREDICTIONS_DIR = OUTPUTS_DIR / 'predictions'


# Paramètres des données
N_CLASSES = config['data']['n_classes']
IMAGES_PER_CLASS = config['data']['images_per_class']
TRAIN_VAL_SPLIT = config['data']['train_val_split']
IMAGE_SIZE = tuple(config['data']['image_size'])
BATCH_SIZE = config['data']['batch_size']
RANDOM_SEED = config['data']['random_seed']
STRATIFIED_SAMPLING = config['data']['stratified_sampling']

# Classes 
CLASSES = config['classes']

# Fixer la graine aléatoire pour la reproductibilité
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)






# RESUMMME

print(f"BASE_DIR           : {BASE_DIR}")
print(f"RAW_DATA_PATH      : {RAW_DATA_PATH}")
print(f"TEST_DATA_PATH     : {TEST_DATA_PATH}")
print(f"BALANCED_DATA_PATH : {BALANCED_DATA_PATH}")
print(f"OUTPUTS_DIR        : {OUTPUTS_DIR}")
print(f"FIGURES_DIR        : {FIGURES_DIR}")
print(f"REPORTS_DIR        : {REPORTS_DIR}")
print()
print(f"N_CLASSES          : {N_CLASSES}")
print(f"IMAGES_PER_CLASS   : {IMAGES_PER_CLASS}")
print(f"IMAGE_SIZE         : {IMAGE_SIZE}")
print(f"BATCH_SIZE         : {BATCH_SIZE}")
print(f"RANDOM_SEED        : {RANDOM_SEED}")
print()
print("CLASSES:")
for i, classe in enumerate(CLASSES, 1):
    print(f"  {i:2d}. {classe}")


BASE_DIR           : A:\Mes documents\CNN
RAW_DATA_PATH      : A:\Mes documents\CNN\data\raw\simpsons_dataset
TEST_DATA_PATH     : A:\Mes documents\CNN\data\raw\kaggle_simpson_testset
BALANCED_DATA_PATH : A:\Mes documents\CNN\data\balanced\simpsons_balanced
OUTPUTS_DIR        : A:\Mes documents\CNN\outputs
FIGURES_DIR        : A:\Mes documents\CNN\outputs\figures
REPORTS_DIR        : A:\Mes documents\CNN\outputs\reports

N_CLASSES          : 13
IMAGES_PER_CLASS   : 850
IMAGE_SIZE         : (224, 224)
BATCH_SIZE         : 32
RANDOM_SEED        : 42

CLASSES:
   1. abraham_grampa_simpson
   2. bart_simpson
   3. charles_montgomery_burns
   4. chief_wiggum
   5. homer_simpson
   6. krusty_the_clown
   7. lisa_simpson
   8. marge_simpson
   9. milhouse_van_houten
  10. moe_szyslak
  11. ned_flanders
  12. principal_skinner
  13. sideshow_bob


# SECTION 2 : ACQUISITION ET VÉRIFICATION DES DONNÉES




### 1. Fonctions utilitaires

In [4]:
def count_images_per_class(data_path, classes):
    
    counts = {}
    
    for class_name in classes:
        class_path = data_path / class_name
        
        if class_path.exists():
            # Utiliser un set pour éviter les doublons
            images = set()
            
            # Ajouter toutes les extensions 
            images.update(class_path.glob("*.jpg"))
            images.update(class_path.glob("*.jpeg"))
            images.update(class_path.glob("*.png"))
            
            counts[class_name] = len(images)
        else:
            print(f"  Dossier manquant: {class_path}")
            counts[class_name] = 0
    
    return counts





# Pour verifier si une image peut être ouverte sans erreur

def verify_image(image_path):
   
    try:
        with Image.open(image_path) as img:
            img.verify()
        return True
    except Exception:
        return False
        


#Recup les dimensions d'une image
def get_image_dimensions(image_path):
    
    try:
        with Image.open(image_path) as img:
            return img.size  # (width, height)
    except Exception:
        return None



# Calculer la luminosité moyenne d'une image
def get_image_brightness(image_path):
   
    try:
        with Image.open(image_path) as img:
            # Convertir en niveaux de gris
            img_gray = img.convert('L')
            # Calculer la moyenne
            return np.mean(np.array(img_gray))
    except Exception:
        return None




### 2. Comptage des images par classe

In [5]:

image_counts = count_images_per_class(RAW_DATA_PATH, CLASSES)

# Affichage des résultats
total_images = 0
for class_name, count in image_counts.items():
    print(f"  {class_name:35s} : {count:5d} images")
    total_images += count

print("-" * 60)
print(f"  {'TOTAL':35s} : {total_images:5d} images")
print()

# Compter les images de test
print("Test set (kaggle_simpson_testset):")
print()
if TEST_DATA_PATH.exists():
    test_images = (list(TEST_DATA_PATH.glob("*.jpg")) + 
                   list(TEST_DATA_PATH.glob("*.jpeg")) + 
                   list(TEST_DATA_PATH.glob("*.png")))
    print(f"  Nombre d'images de test : {len(test_images):5d} images")
else:
    print("    Dossier de test pas  trouvé")
    test_images = []

print()

  abraham_grampa_simpson              :   913 images
  bart_simpson                        :  1342 images
  charles_montgomery_burns            :  1193 images
  chief_wiggum                        :   986 images
  homer_simpson                       :  2246 images
  krusty_the_clown                    :  1206 images
  lisa_simpson                        :  1354 images
  marge_simpson                       :  1291 images
  milhouse_van_houten                 :  1079 images
  moe_szyslak                         :  1452 images
  ned_flanders                        :  1454 images
  principal_skinner                   :  1194 images
  sideshow_bob                        :   877 images
------------------------------------------------------------
  TOTAL                               : 16587 images

Test set (kaggle_simpson_testset):

  Nombre d'images de test :   641 images



### 3. Détection des images corrompues

In [6]:

corrupted_images = []

for class_name in tqdm(CLASSES, desc="Vérification des classes"):
    class_path = RAW_DATA_PATH / class_name
    
    if class_path.exists():
        images = (list(class_path.glob("*.jpg")) + 
                 list(class_path.glob("*.jpeg")) + 
                 list(class_path.glob("*.png")))
        
        for img_path in images:
            if not verify_image(img_path):
                corrupted_images.append(str(img_path))

print()
if len(corrupted_images) == 0:
    print(" Aucune image corrompue détectée ")
else:
    print(f"  {len(corrupted_images)} image(s) corrompue(s) détectée(s)")
    for img in corrupted_images[:5]:
        print(f"     - {img}")
    if len(corrupted_images) > 5:
        print(f"     ... et {len(corrupted_images) - 5} autre(s)")

print()

Vérification des classes: 100%|██████████| 13/13 [03:26<00:00, 15.91s/it]


 Aucune image corrompue détectée 



### 4. Statistiques globales

In [7]:

counts_values = list(image_counts.values())
min_images = min(counts_values)
max_images = max(counts_values)
mean_images = np.mean(counts_values)
median_images = np.median(counts_values)
std_images = np.std(counts_values)

print(f"Nombre total d'images    : {sum(counts_values)}")
print(f"Nombre de classes        : {len(CLASSES)}")
print()
print("Distribution par classe:")
print(f"  - Minimum              : {min_images} images ({[k for k, v in image_counts.items() if v == min_images][0]})")
print(f"  - Maximum              : {max_images} images ({[k for k, v in image_counts.items() if v == max_images][0]})")
print(f"  - Moyenne              : {mean_images:.1f} images")
print(f"  - Médiane              : {median_images:.1f} images")
print(f"  - Écart-type           : {std_images:.1f} images")
print()
print(f"Ratio max/min            : {max_images/min_images:.2f}x")
print(f"Images corrompues        : {len(corrupted_images)}")
print()

Nombre total d'images    : 16587
Nombre de classes        : 13

Distribution par classe:
  - Minimum              : 877 images (sideshow_bob)
  - Maximum              : 2246 images (homer_simpson)
  - Moyenne              : 1275.9 images
  - Médiane              : 1206.0 images
  - Écart-type           : 333.6 images

Ratio max/min            : 2.56x
Images corrompues        : 0



### 5. Sauvegarde 

In [8]:
data_report = pd.DataFrame([
    {'Classe': classe, 'Nombre_Images': count}
    for classe, count in image_counts.items()
])
data_report.loc[len(data_report)] = ['TOTAL', sum(counts_values)]

report_path = REPORTS_DIR / 'data_report.csv'
data_report.to_csv(report_path, index=False, encoding='utf-8')

print(f" Rapport sauvegardé dans : {report_path}")


 Rapport sauvegardé dans : A:\Mes documents\CNN\outputs\reports\data_report.csv


# SECTION 3 : ANALYSE EXPLORATOIRE DES DONNÉES (EDA)




In [9]:


# Créer le dossier pour les figures EDA
EDA_FIGURES_DIR = FIGURES_DIR / 'eda'
EDA_FIGURES_DIR.mkdir(parents=True, exist_ok=True)

### 1. Visualisation de la distribution des classes

In [10]:

plt.figure(figsize=(10, 8))

classes_names = list(image_counts.keys())
classes_counts = list(image_counts.values())

bars = plt.bar(range(len(classes_names)), classes_counts, 
               color='steelblue', alpha=0.8, edgecolor='navy', linewidth=1.5)

# Ajout de valeurs
for bar, value in zip(bars, classes_counts):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 30,
            f'{value}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Ligne de moyenne
plt.axhline(y=mean_images, color='red', linestyle='--', linewidth=2, 
            label=f'Moyenne: {mean_images:.0f} images', alpha=0.7)

plt.xlabel('Classes (Personnages)', fontsize=14, fontweight='bold')
plt.ylabel('Nombre d\'images', fontsize=14, fontweight='bold')
plt.title('Distribution des Images par Classe (Dataset pas encore exploité)', 
          fontsize=16, fontweight='bold', pad=20)
plt.xticks(range(len(classes_names)), classes_names, rotation=45, ha='right', fontsize=10)
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.legend(fontsize=12)
plt.tight_layout()

save_path = EDA_FIGURES_DIR / '01_distribution_classes.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f" Graphique sauvegardé dans: {save_path}")
plt.close()

print()

 Graphique sauvegardé dans: A:\Mes documents\CNN\outputs\figures\eda\01_distribution_classes.png



### 2.  Analyse des dimensions des images

In [11]:

widths = []
heights = []
aspects = []

for class_name in tqdm(CLASSES, desc="Analyse des dimensions"):
    class_path = RAW_DATA_PATH / class_name
    
    if class_path.exists():
        images = (list(class_path.glob("*.jpg")) + 
                 list(class_path.glob("*.jpeg")) + 
                 list(class_path.glob("*.png")))
        
        # Échantillonner max 100 images par classe pour l'analyse   
        sample_images = random.sample(images, min(100, len(images)))
        
        for img_path in sample_images:
            dims = get_image_dimensions(img_path)
            if dims:
                w, h = dims
                widths.append(w)
                heights.append(h)
                aspects.append(w / h if h > 0 else 1.0)

print()
print(f"Analyse effectuée sur {len(widths)} images")
print()
print("Statistiques des dimensions:")
print(f"  Largeur  - Min: {min(widths):4d}px, Max: {max(widths):4d}px, Moyenne: {np.mean(widths):.1f}px")
print(f"  Hauteur  - Min: {min(heights):4d}px, Max: {max(heights):4d}px, Moyenne: {np.mean(heights):.1f}px")
print(f"  Ratio L/H - Min: {min(aspects):.2f}, Max: {max(aspects):.2f}, Moyenne: {np.mean(aspects):.2f}")
print()

# Créer le graphique des dimensions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Largeur
axes[0].hist(widths, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(np.mean(widths), color='red', linestyle='--', linewidth=2, label=f'Moyenne: {np.mean(widths):.0f}px')
axes[0].set_xlabel('Largeur (pixels)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Nombre d\'images', fontsize=12, fontweight='bold')
axes[0].set_title('Largeurs', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Hauteur
axes[1].hist(heights, bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1].axvline(np.mean(heights), color='red', linestyle='--', linewidth=2, label=f'Moyenne: {np.mean(heights):.0f}px')
axes[1].set_xlabel('Hauteur (pixels)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Nombre d\'images', fontsize=12, fontweight='bold')
axes[1].set_title(' Hauteurs', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

# Ratio
axes[2].hist(aspects, bins=50, color='lightgreen', edgecolor='black', alpha=0.7)
axes[2].axvline(np.mean(aspects), color='red', linestyle='--', linewidth=2, label=f'Moyenne: {np.mean(aspects):.2f}')
axes[2].set_xlabel('Ratio Largeur/Hauteur', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Nombre d\'images', fontsize=12, fontweight='bold')
axes[2].set_title('Distribution des Ratios', fontsize=14, fontweight='bold')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
save_path = EDA_FIGURES_DIR / '02_distribution_dimensions.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f" Graphique sauvegardé dans: {save_path}")
plt.close()

print()

Analyse des dimensions: 100%|██████████| 13/13 [00:00<00:00, 34.16it/s]



Analyse effectuée sur 1300 images

Statistiques des dimensions:
  Largeur  - Min:  256px, Max:  956px, Moyenne: 401.8px
  Hauteur  - Min:  256px, Max: 1072px, Moyenne: 415.6px
  Ratio L/H - Min: 0.49, Max: 1.91, Moyenne: 0.98

 Graphique sauvegardé dans: A:\Mes documents\CNN\outputs\figures\eda\02_distribution_dimensions.png



### 3. Analyse de la luminosité

In [12]:

brightnesses = []

for class_name in tqdm(CLASSES, desc="Analyse de la luminosité"):
    class_path = RAW_DATA_PATH / class_name
    
    if class_path.exists():
        images = (list(class_path.glob("*.jpg")) + 
                 list(class_path.glob("*.jpeg")) + 
                 list(class_path.glob("*.png")))
        
        sample_images = random.sample(images, min(100, len(images)))
        
        for img_path in sample_images:
            brightness = get_image_brightness(img_path)
            if brightness is not None:
                brightnesses.append(brightness)

print()
print(f"Analyse effectuée sur {len(brightnesses)} images")
print()
print("Statistiques de luminosité:")
print(f"  Min     : {min(brightnesses):.1f}")
print(f"  Max     : {max(brightnesses):.1f}")
print(f"  Moyenne : {np.mean(brightnesses):.1f}")
print(f"  Médiane : {np.median(brightnesses):.1f}")
print(f"  Écart-type: {np.std(brightnesses):.1f}")
print()

plt.figure(figsize=(12, 6))
plt.hist(brightnesses, bins=50, color='gold', edgecolor='black', alpha=0.7)
plt.axvline(np.mean(brightnesses), color='red', linestyle='--', linewidth=2, 
            label=f'Moyenne: {np.mean(brightnesses):.1f}')
plt.axvline(np.median(brightnesses), color='blue', linestyle='--', linewidth=2, 
            label=f'Médiane: {np.median(brightnesses):.1f}')
plt.xlabel('Luminosité moyenne [0-255]', fontsize=12, fontweight='bold')
plt.ylabel('Nombre d\'images', fontsize=12, fontweight='bold')
plt.title('Luminosité des Images', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()

save_path = EDA_FIGURES_DIR / '03_distribution_luminosite.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f" Graphique sauvegardé dans: {save_path}")
plt.close()

print()

Analyse de la luminosité: 100%|██████████| 13/13 [00:04<00:00,  2.81it/s]



Analyse effectuée sur 1300 images

Statistiques de luminosité:
  Min     : 36.8
  Max     : 240.3
  Moyenne : 108.2
  Médiane : 107.5
  Écart-type: 29.6

 Graphique sauvegardé dans: A:\Mes documents\CNN\outputs\figures\eda\03_distribution_luminosite.png



### 4. Visualisation d'échantillons aléatoires

In [13]:

fig, axes = plt.subplots(13, 5, figsize=(15, 35))
fig.suptitle('Échantillons Aléatoires par Classe (5 images/classe)', 
             fontsize=16, fontweight='bold', y=0.995)

for idx, class_name in enumerate(CLASSES):
    class_path = RAW_DATA_PATH / class_name
    
    if class_path.exists():
        images = (list(class_path.glob("*.jpg")) + 
                 list(class_path.glob("*.jpeg")) + 
                 list(class_path.glob("*.png")))
        
        sample_images = random.sample(images, min(5, len(images)))
        
        for col, img_path in enumerate(sample_images):
            try:
                img = Image.open(img_path)
                axes[idx, col].imshow(img)
                axes[idx, col].axis('off')
                if col == 0:
                    axes[idx, col].set_title(class_name, fontsize=10, loc='left')
            except:
                axes[idx, col].axis('off')

plt.tight_layout()
save_path = EDA_FIGURES_DIR / '04_echantillons_par_classe.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f" Graphique sauvegardé dans: {save_path}")
plt.close()



 Graphique sauvegardé dans: A:\Mes documents\CNN\outputs\figures\eda\04_echantillons_par_classe.png


# SECTION 4 : EQUILIBRAGE STRATIFIÉ DU DATASET




In [14]:

print(f"Objectif : Sélectionner {IMAGES_PER_CLASS} images par classe")
print(f"Stratégie : {'Stratified Random Sampling' if STRATIFIED_SAMPLING else 'Random Sampling'}")
print()

Objectif : Sélectionner 850 images par classe
Stratégie : Stratified Random Sampling



### 1. LA Fonction d'équilibrage 

In [15]:
def stratified_sampling(images_list, n_samples, brightness_bins=4):
   
    if len(images_list) <= n_samples:
        return images_list
    
    # Calculer la luminosité pour toutes les images
    print("      Calcul des luminosités...")
    brightnesses_dict = {}
    for img_path in tqdm(images_list, desc="      ", leave=False):
        brightness = get_image_brightness(img_path)
        if brightness is not None:
            brightnesses_dict[img_path] = brightness
    
    # Créer des bins de luminosité
    valid_images = list(brightnesses_dict.keys())
    valid_brightnesses = list(brightnesses_dict.values())
    
    # Diviser en bins
    bins = np.linspace(min(valid_brightnesses), max(valid_brightnesses), brightness_bins + 1)
    bin_indices = np.digitize(valid_brightnesses, bins) - 1
    
    # Échantillonner proportionnellement dans chaque bin
    selected_images = []
    samples_per_bin = n_samples // brightness_bins
    remaining = n_samples % brightness_bins
    
    for bin_idx in range(brightness_bins):
        bin_images = [img for img, b_idx in zip(valid_images, bin_indices) if b_idx == bin_idx]
        
        n_to_sample = samples_per_bin + (1 if bin_idx < remaining else 0)
        n_to_sample = min(n_to_sample, len(bin_images))
        
        selected_images.extend(random.sample(bin_images, n_to_sample))
    
    # Si on n'a pas assez,on le  compléter aléatoirement
    if len(selected_images) < n_samples:
        remaining_images = [img for img in valid_images if img not in selected_images]
        additional = random.sample(remaining_images, n_samples - len(selected_images))
        selected_images.extend(additional)
    
    return selected_images[:n_samples]

### 2. Équilibrage et copie des images

In [16]:

# Création du dossier pour le dataset équilibré
BALANCED_DATA_PATH.mkdir(parents=True, exist_ok=True)

balanced_counts = {}

for class_name in CLASSES:
    print(f"Traitement de la classe: {class_name}")
    
    # Chemin source et destination
    source_class_path = RAW_DATA_PATH / class_name
    dest_class_path = BALANCED_DATA_PATH / class_name
    dest_class_path.mkdir(parents=True, exist_ok=True)
    
    if source_class_path.exists():
        # Récupération de toutes les images 
        all_images = (list(source_class_path.glob("*.jpg")) + 
                     list(source_class_path.glob("*.jpeg")) + 
                     list(source_class_path.glob("*.png")))
        
        print(f"   Images disponibles: {len(all_images)}")
        
        # Sélection des images 
        if STRATIFIED_SAMPLING and len(all_images) > IMAGES_PER_CLASS:
            selected_images = stratified_sampling(all_images, IMAGES_PER_CLASS)
        else:
            # Random sampling simple
            n_to_sample = min(IMAGES_PER_CLASS, len(all_images))
            selected_images = random.sample(all_images, n_to_sample)
        
        print(f"   Images sélectionnées: {len(selected_images)}")
        
        # Copie des images sélectionnées
        for img_path in tqdm(selected_images, desc="      Copie", leave=False):
            dest_path = dest_class_path / img_path.name
            if not dest_path.exists():
                shutil.copy2(img_path, dest_path)
        
        balanced_counts[class_name] = len(selected_images)
        print(f"    Terminé")
    else:
        print(f"     Dossier source introuvable")
        balanced_counts[class_name] = 0
    
    print()

Traitement de la classe: abraham_grampa_simpson
   Images disponibles: 913
      Calcul des luminosités...


   Images sélectionnées: 850


    Terminé

Traitement de la classe: bart_simpson
   Images disponibles: 1342
      Calcul des luminosités...


   Images sélectionnées: 850


    Terminé

Traitement de la classe: charles_montgomery_burns
   Images disponibles: 1193
      Calcul des luminosités...


...

    Terminé

Traitement de la classe: principal_skinner
   Images disponibles: 1194
      Calcul des luminosités...


   Images sélectionnées: 850


    Terminé

Traitement de la classe: sideshow_bob
   Images disponibles: 877
      Calcul des luminosités...


   Images sélectionnées: 850


    Terminé



### 3. Vérification du dataset équilibré

In [17]:

print("Nombre d'images par classe:")
print()

total_balanced = 0
for class_name, count in balanced_counts.items():
    print(f"  {class_name:35s} : {count:4d} images")
    total_balanced += count

print("-" * 60)
print(f"  {'TOTAL':35s} : {total_balanced:4d} images")
print()



# Calculer le nombre d'images pour train et validation
n_train = int(total_balanced * TRAIN_VAL_SPLIT)
n_val = total_balanced - n_train

print(f"Split train/validation ({int(TRAIN_VAL_SPLIT*100)}/{int((1-TRAIN_VAL_SPLIT)*100)}):")
print(f"  Train      : {n_train:5d} images ({int(n_train/N_CLASSES)} par classe)")
print(f"  Validation : {n_val:5d} images ({int(n_val/N_CLASSES)} par classe)")
print()

Nombre d'images par classe:

  abraham_grampa_simpson              :  850 images
  bart_simpson                        :  850 images
  charles_montgomery_burns            :  850 images
  chief_wiggum                        :  850 images
  homer_simpson                       :  850 images
  krusty_the_clown                    :  850 images
  lisa_simpson                        :  850 images
  marge_simpson                       :  850 images
  milhouse_van_houten                 :  850 images
  moe_szyslak                         :  850 images
  ned_flanders                        :  850 images
  principal_skinner                   :  850 images
  sideshow_bob                        :  850 images
------------------------------------------------------------
  TOTAL                               : 11050 images

Split train/validation (80/19):
  Train      :  8840 images (680 par classe)
  Validation :  2210 images (170 par classe)



### 4. Visualisation du dataset équilibré

In [18]:

plt.figure(figsize=(10, 8))

classes_names = list(balanced_counts.keys())
classes_counts = list(balanced_counts.values())

bars = plt.bar(range(len(classes_names)), classes_counts, 
               color='seagreen', alpha=0.8, edgecolor='darkgreen', linewidth=1.5)

# Ajouter les valeurs
for bar, value in zip(bars, classes_counts):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
            f'{value}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Ligne de cible
plt.axhline(y=IMAGES_PER_CLASS, color='red', linestyle='--', linewidth=2, 
            label=f'Cible: {IMAGES_PER_CLASS} images', alpha=0.7)

plt.xlabel('Classes (Personnages)', fontsize=14, fontweight='bold')
plt.ylabel('Nombre d\'images', fontsize=14, fontweight='bold')
plt.title('Distribution des Images par Classe (celui du Dataset Équilibré)', 
          fontsize=16, fontweight='bold', pad=20)
plt.xticks(range(len(classes_names)), classes_names, rotation=45, ha='right', fontsize=10)
plt.ylim([0, IMAGES_PER_CLASS + 50])
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.legend(fontsize=12)
plt.tight_layout()

save_path = EDA_FIGURES_DIR / '05_distribution_equilibree.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f" Graphique sauvegardé dans : {save_path}")
plt.close()

print()

 Graphique sauvegardé dans : A:\Mes documents\CNN\outputs\figures\eda\05_distribution_equilibree.png



#### 5 - Sauvegarde 

In [19]:
balanced_report = pd.DataFrame([
    {
        'Classe': classe, 
        'Images_Brutes': image_counts[classe],
        'Images_Equilibrees': balanced_counts[classe],
        'Difference': balanced_counts[classe] - image_counts[classe]
    }
    for classe in CLASSES
])

balanced_report.loc[len(balanced_report)] = [
    'TOTAL',
    sum(image_counts.values()),
    sum(balanced_counts.values()),
    sum(balanced_counts.values()) - sum(image_counts.values())
]

report_path = REPORTS_DIR / 'balanced_data_report.csv'
balanced_report.to_csv(report_path, index=False, encoding='utf-8')

print(f" Rapport d'équilibrage sauvegardé dans : {report_path}")
print()
print("Aperçu du rapport:")
print(balanced_report.to_string(index=False))
print()



 Rapport d'équilibrage sauvegardé dans : A:\Mes documents\CNN\outputs\reports\balanced_data_report.csv

Aperçu du rapport:
                  Classe  Images_Brutes  Images_Equilibrees  Difference
  abraham_grampa_simpson            913                 850         -63
            bart_simpson           1342                 850        -492
charles_montgomery_burns           1193                 850        -343
            chief_wiggum            986                 850        -136
           homer_simpson           2246                 850       -1396
        krusty_the_clown           1206                 850        -356
            lisa_simpson           1354                 850        -504
           marge_simpson           1291                 850        -441
     milhouse_van_houten           1079                 850        -229
             moe_szyslak           1452                 850        -602
            ned_flanders           1454                 850        -604
       princi